# Data For Social Justice Methodology
Each header in this notebook corresponds to a headers in the methodology.

In [1]:
import os
import pandas as pd

In [2]:
# inputs
fn_policy = '../data/output/placements_api_keyword_status/policy.csv'
fn_hate = '../data/output/placements_api_keyword_status/hate.csv'
fn_adhoc = '../data/output/placements_api_keyword_status/adhoc.csv'
fn_basewords = '../data/output/placements_api_keyword_status/basewords.csv'
fn_social_justice = '../data/output/placements_api_keyword_status/social_justice.csv'

# outputs
table_dir = '../data/output/tables/socialjustice'
fn_table1 = '../data/output/tables/socialjustice/table1.csv'
os.makedirs(table_dir, exist_ok=True)

In [3]:
df_soc_just = pd.read_csv(fn_social_justice)
df_adhoc = pd.read_csv(fn_adhoc)
df_policy = pd.read_csv(fn_policy)
df_basewords = pd.read_csv(fn_basewords)
df_hate = pd.read_csv(fn_hate)

In [4]:
display_cols = ['search_term', 'status']

## What is Blocked?

In [5]:
col2social_justice_table_col = {
    'search_term': "Search Term", 
    'status': "Statusfor Ad Placement",
    'status_no_spaces': "Status with Spaces Removed",
    'n_youtube_videos': "N YouTube Videos",
    'n_youtube_channels': "N YouTube Videos",
}

In [6]:
social_justice_table = df_soc_just[[c for c in col2social_justice_table_col.keys()]]
social_justice_table.columns = col2social_justice_table_col.values()
social_justice_table.to_csv(fn_table1)

In [7]:
df_soc_just.status.value_counts()

Full             42
Blocked          17
Partial Block     3
Name: status, dtype: int64

In [8]:
df_soc_just.status.value_counts(normalize=True)

Full             0.677419
Blocked          0.274194
Partial Block    0.048387
Name: status, dtype: float64

## White power vs Black power

In [9]:
df_soc_just[df_soc_just.search_term == 'black power'].append(
    df_hate[df_hate.search_term == 'white power']
)[display_cols]

,search_term,status
58,black power,Blocked
1,white power,Full


In [10]:
df_soc_just[df_soc_just.search_term.str.contains('matter')].append(
    df_hate[df_hate.search_term == 'white lives matter']
).append(
    df_adhoc[df_adhoc.search_term.str.contains('matter')]
)[display_cols]

,search_term,status
10,black girls matter,Full
30,black trans lives matter,Full
60,black lives matter,Blocked
10,white lives matter,Full
6,all lives matter,Full
13,blue lives matter,Full


In [11]:
df_soc_just[df_soc_just.search_term.str.contains('muslim')][
    ['search_term', 'status']
]

,search_term,status
50,no muslim ban ever,Blocked
51,muslim solidarity,Blocked
52,muslim parenting,Blocked
53,muslim fashion,Blocked
54,muslim american,Blocked
59,black muslim,Blocked
61,american muslim,Blocked


In [12]:
df_soc_just[df_soc_just.search_term.str.contains('muslim fashion')].append(
    df_adhoc[df_adhoc.search_term.str.contains('fashion')]
)[['search_term', 'status']]

,search_term,status
53,muslim fashion,Blocked
15,christian fashion,Full
29,jewish fashion,Full
38,buddhist fashion,Full


In [13]:
df_soc_just[df_soc_just.search_term.str.contains('muslim parenting')].append(
    df_adhoc[df_adhoc.search_term.str.contains('parenting')]
)[['search_term', 'status']]

,search_term,status
52,muslim parenting,Blocked
22,christian parenting,Full
69,jewish parenting,Blocked
73,buddhist parenting,Blocked


In [14]:
df_soc_just[df_soc_just.status == 'Blocked'].status_no_spaces.value_counts()

Full             13
Blocked           3
Partial Block     1
Name: status_no_spaces, dtype: int64

In [15]:
df_soc_just[df_soc_just.status == 'Blocked'][['search_term', 'status', 'status_no_spaces']]

,search_term,status,status_no_spaces
45,sexual liberation,Blocked,Blocked
46,sex work,Blocked,Blocked
47,sex ed,Blocked,Blocked
48,white supremacy,Blocked,Full
49,stand with ilhan,Blocked,Full
50,no muslim ban ever,Blocked,Full
51,muslim solidarity,Blocked,Full
52,muslim parenting,Blocked,Full
53,muslim fashion,Blocked,Full
54,muslim american,Blocked,Full


In [16]:
df_hate[df_hate.search_term.str.contains('sharia|jihad|radical islamic terror')]

,search_term,status,status_no_spaces,n_youtube_videos,n_youtube_channels,n_youtube_videos_no_spaces,n_youtube_channels_no_spaces,additional_info_link
33,white sharia,Full,NaN,383290.0,14.0,NaN,NaN,https://www.splcenter.org/hatewatch/2017/11/27...
40,civilization jihad,Full,NaN,79645.0,5.0,NaN,NaN,https://www.splcenter.org/fighting-hate/extrem...
68,radical islamic terror,Blocked,Full,NaN,NaN,1746.0,0.0,http://csps.gmu.edu/2020/12/01/why-does-the-we...


## Band-Aids

In [17]:
df_policy[df_policy.search_term.str.contains('covid-19')].append(
    df_basewords[df_basewords.search_term.str.contains('covid-19')]
)[['search_term', 'status']]

,search_term,status
57,covid-19 vaccine kills people,Blocked
58,covid-19 ritual,Blocked
59,covid-19 prayer,Blocked
60,covid-19 people don't die,Blocked
61,covid-19 home remedy,Blocked
62,covid-19 guaranteed vaccine,Blocked
63,covid-19 from 5g,Blocked
64,covid-19 doesn't exist,Blocked
65,covid-19 asian food,Blocked
138,covid-19,Blocked


In [18]:
df_policy[df_policy.search_term.str.contains('sex')].append(
    df_basewords[df_basewords.search_term.str.contains('sex')]
)[['search_term', 'status']]

,search_term,status
39,sexually unwanted acts,Blocked
40,sexually explict content,Blocked
41,sexual harassment,Blocked
42,sexual gratification,Blocked
43,sexual fluids,Blocked
44,sexual fetishes,Blocked
45,sexual device,Blocked
46,sexual acts,Blocked
47,sexual abuse,Blocked
48,sex toys,Blocked
